In [ ]:
# Input for anchor, positive, negative

in_array = Input(shape=(96, 96, 3), name="img_a")

# create base model
model = create_model()

# Output embedding vectors from anchor, positive, negative
# Model wegihts are shared (Triplet)
model_in = model(in_array)

# layer that computes the triplet loss from anchor, positive, and negative embedding vectors
loss_function = LossLayer(arguments)([model_in])

# Model that can be trained with anchor, positive, negative images
model = Model([in_array], loss_function)

# load weights
# model.load_weights('ckpts/coml-weights-aug6.hdf5'

In [ ]:
# Initialize the Preprocessor

batchsize = 10
generator = DataGenerator(arguments)

In [ ]:
# Run the training
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 100
generator.batch_size = 16

model.compile(loss=None, optimizer='adam')
with tf.device('/gpu:0'):
    history = model.fit_generator(generator=generator,
                                  epochs=EPOCHS,
                                  callbacks=[ModelCheckpoint(filepath='ckpts/epoch{epoch:03d}_loss{loss:.3f}.hdf5',
                                                             monitor='loss',
                                                             save_best_only=False,
                                                             mode='auto',
                                                             save_weights_only=True,
                                                             verbose=1),
                                             EarlyStopping(monitor='loss',
                                                           mode='auto',
                                                           patience=10,
                                                           verbose=True
                                                           )])